# Packages

In [1]:
import numpy as np
import pandas as pd
import requests

# Secrets
You will need a Google API key stored in the file referenced below to actually run the API calls.  I will provide a copy of the file output by this code.

In [2]:
with open('../secrets_dva_project.txt') as f:
    API_KEY = f.read().strip()

# Variables

In [3]:
dataFolderName = "UberData"    # The name of the subfolder the data is stored in.  Used to construct relative file path.
debug = True # Do you want debugging messages?

# API Call Function

In [4]:
def get_address_from_lat_lng(latitude, longitude):
    response = requests.get(f'https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={API_KEY}')

    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            #return data['results'][0]['formatted_address']
            return data
    else:
        print("Non 200 Status:")
        print("Code:", response.status_code)
        print("Reason:", response.reason)

    return None            

# API Results Parse Function

In [5]:
def parse_address(data):
    addrDict = {}

    haveStreetNo = False
    haveStreetNm = False
    haveLocality = False
    haveCounty   = False
    haveState    = False
    haveCountry  = False
    havePostal   = False
    
    if 'results' in data and len(data['results']) > 0:
        #return data['results'][0]['formatted_address']
        #return data['results'][0]['address_components']

        # loop the results
        for res in data['results']:
            # loop address_components
            # If we already have this component type, skip it
            for comp in res['address_components']:
                if 'street_number' in comp['types'] and not haveStreetNo:
                    if debug: print('street_number:', comp["short_name"])
                    addrDict["street_number"] = comp["short_name"]
                    haveStreetNo = True

                if 'route' in comp['types'] and not haveStreetNm:
                    if debug: print('street_name:', comp["short_name"])
                    addrDict["street_name"] = comp["short_name"]
                    haveStreetNm = True

                if 'locality' in comp['types'] and not haveLocality:
                    if debug: print('locality:', comp["short_name"])
                    addrDict["locality"] = comp["short_name"]
                    haveLocality = True

                if 'administrative_area_level_2' in comp['types'] and not haveCounty:
                    if debug: print('county:', comp["short_name"])
                    addrDict["county"] = comp["short_name"]
                    haveCounty = True

                if 'administrative_area_level_1' in comp['types'] and not haveState:
                    if debug: print('state:', comp["long_name"], comp["short_name"])
                    addrDict["state_short"] = comp["short_name"]
                    addrDict["state_long"] = comp["long_name"]
                    haveState = True

                if 'country' in comp['types'] and not haveCountry:
                    if debug: print('country:', comp["long_name"], comp["short_name"])
                    addrDict["country"] = comp["short_name"]
                    addrDict["country"] = comp["long_name"]
                    haveCountry = True

                if 'postal_code' in comp['types'] and not havePostal:
                    if debug: print('postal_code:', comp["short_name"])
                    addrDict["postal_code"] = comp["short_name"]
                    havePostal = True
        
            # If we have them all, break the loop
            if haveStreetNo and haveStreetNm and haveLocality and haveCounty and haveState and haveCountry and havePostal:
                break

        return addrDict
    
    return None

# Example Call

In [6]:

# Replace with your latitude and longitude
latitude = 37.4219999
longitude = -122.0840575

address = parse_address(get_address_from_lat_lng(latitude, longitude))
if address is not None:
    print('The address corresponding to the given latitude and longitude is:' + "\n" + \
          f'{address["street_number"]} {address["street_name"]}' + '\n' + \
          f'{address["locality"]}, {address["state_short"]} {address["postal_code"]}')
else:
    print('Could not find an address for the given latitude and longitude')

street_number: P1600
street_name: Amphitheatre Pkwy
locality: Mountain View
county: Santa Clara County
state: California CA
country: United States US
postal_code: 94043
The address corresponding to the given latitude and longitude is:
P1600 Amphitheatre Pkwy
Mountain View, CA 94043


# File Read

In [7]:
fileName = "data_location_20231128_200050.csv"
#filePath = dataFolderName +"\\"+ fileName
filePath = fileName
df_latlon = pd.read_csv(filePath)
df_latlon = df_latlon[["idloc","lat","lon"]]

In [8]:
df_latlon

,idloc,lat,lon
0,1,33.754193,-84.066023
1,76,33.835545,-84.158713


# Process File

In [9]:
debug = False

names = list()

for row in df_latlon.itertuples(index=False):
    if debug and row.idloc > 5: break

    if debug: print(row)
    address = parse_address(get_address_from_lat_lng(row.lat, row.lon))
    addr = {}
    addr["idloc"] = row.idloc
    #if 'street_number' in address: addr['street_number'] = address['street_number'] else: addr['street_number'] = ''
    
    addr["street_number"] = address.get("street_number","")
    addr["street_name"] = address.get("street_name","")
    addr["locality"] = address.get("locality","")
    addr["state_short"] = address.get("state_short","")
    addr["state_long"] = address.get("state_long","")
    addr["postal_code"] = address.get("postal_code","")
    
    names.append(addr)

if debug: print(names)

df_names = pd.DataFrame(names)
if debug: print(df_names)

In [10]:
df_names

,idloc,street_number,street_name,locality,state_short,state_long,postal_code
0,1,1990,Rock Mountain Rd,Stonecrest,GA,Georgia,30058
1,76,1829,Tilling Way,Stone Mountain,GA,Georgia,30087


In [11]:
# write file
from datetime import datetime
strnow = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = "data_names_" + strnow + ".csv"
df_names.to_csv(filename, index=False)

# Test Function Call

In [14]:
lat = 33.7541928
lon = -84.0660228
dat = get_address_from_lat_lng(lat,lon)
res = parse_address(dat)
display(res)

{'locality': 'Stonecrest',
 'county': 'Dekalb County',
 'state_short': 'GA',
 'state_long': 'Georgia',
 'country': 'United States',
 'street_name': 'Rock Mountain Rd',
 'postal_code': '30058',
 'street_number': '1990'}